# Movies Analyis Project

## Introduction




This project analyzes IMDB(Internet Movie Database) database and a Movie Budgets Dataset to identify what types of films perform best. 

The goal  is to provide data-driven insights  on `movie genres`, `production budgets` and `release timing`that will guide the launch of a new movie studio for our company.


## Objectives of the Project



1. Clean and prepare the movie datasets
2. Explore and visualize movie trends
3. Recommend profitable genres and strategies for the new studio


## Business Understanding

The company’s management wants to enter the movie production industry but lacks experience in identifying what types of films are most likely to succeed.

This analysis aims to support management by answering the following key business questions:

  1. Which movie genres are the most profitable and well-received by audiences?

  2. Which genres perform well despite lower production budgets?

  3. Does the timing of a movie’s release impact its box office performance?





## Data Understanding

The analysis uses 2 datasets (IMDB Database and Movie Budgets dataset)that provide complementary information about movies:

`The IMDB Database` contains:
 - Movie Basics Table: Contains movie titles, genres, and release dates.

 - Movie Ratings Table: Includes average audience ratings and number of votes for each movie.

`The Movie Budgets Dataset`:
 Provides production budgets and the revenue figures.

These datasets  will be merged to create a unified view of each movie’s genre, financial performance, audience reception, and release timing.

## Loading the data into dataframes

In [2]:
# importing the necessary libraries

import pandas as pd
import numpy as np
import sqlite3

#### Movie Budgets Dataset

In [3]:
#Creating a movie budgets dataframe
movie_budgets = pd.read_csv("./original_data/tn.movie_budgets.csv.gz")
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


#### The IMDB Dataset

In [6]:
import zipfile
import pandas as pd

# unzip the database
with zipfile.ZipFile('original_data/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall()

# connect to the SQLite database
conn = sqlite3.connect('im.db')


In [50]:
query = """
SELECT
    mb.movie_id,
    mb.primary_title,
    mb.genres,
    mb.start_year,
    mb.runtime_minutes,
    mr.averagerating,
    mr.numvotes
FROM movie_basics AS mb
JOIN movie_ratings AS mr
    ON mb.movie_id = mr.movie_id
WHERE mb.start_year IS NOT NULL
  AND mb.genres IS NOT NULL;
"""
movie_database = pd.read_sql_query(query, conn)

#Close the connection
#conn.close()

#Display first few rows of the movies_database dataframe
movie_database.head()



,movie_id,primary_title,genres,start_year,runtime_minutes,averagerating,numvotes
0,tt0063540,Sunghursh,"Action,Crime,Drama",2013,175.0,7.0,77
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",2019,114.0,7.2,43
2,tt0069049,The Other Side of the Wind,Drama,2018,122.0,6.9,4517
3,tt0069204,Sabse Bada Sukh,"Comedy,Drama",2018,NaN,6.1,13
4,tt0100275,The Wandering Soap Opera,"Comedy,Drama,Fantasy",2017,80.0,6.5,119


### Cleaning Movie Budget Dataset

In [ ]:
#check movie budget structure
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [ ]:
#check for null values
# no null values are in the movie_budget dataset and all columns have data
movie_budgets.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [ ]:
#check if there is any duplicate
#no duplicates in movie_budget dataset
movie_budgets.duplicated().sum()

0

In [52]:
#change production_budget,domestic_gross and worldwide_gross to float
movie_budgets.head()
#this columns are to be converted in floats
movie_budgets['production_budget']=movie_budgets['production_budget'].str.replace(',','').str.replace('$','').astype(float)
movie_budgets['domestic_gross']=movie_budgets['domestic_gross'].str.replace(',','').str.replace('$','').astype(float)
movie_budgets['worldwide_gross']=movie_budgets['worldwide_gross'].str.replace(',','').str.replace('$','').astype(float)

<ipython-input-52-e24b2fc9078c>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_budgets['production_budget']=movie_budgets['production_budget'].str.replace(',','').str.replace('$','').astype(float)
<ipython-input-52-e24b2fc9078c>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_budgets['domestic_gross']=movie_budgets['domestic_gross'].str.replace(',','').str.replace('$','').astype(float)
<ipython-input-52-e24b2fc9078c>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_budgets['worldwide_gross']=movie_bu

In [68]:
#change release_date to date
movie_budgets['release_date']=pd.to_datetime(movie_budgets['release_date'],dayfirst=True)
movie_budgets['release_date']=movie_budgets['release_date'].dt.strftime('%d-%m-%Y')

In [ ]:
#rename column movie column to movie_title and drop id column
movie_budgets.rename(columns={'movie':'movie_title'},inplace=True)
movie_budgets.drop(columns='id',inplace=True)
movie_budgets.head()

,release_date,movie_title,production_budget,domestic_gross,worldwide_gross
0,18-12-2009,Avatar,425000000.0,760507625.0,2.776345e+09
1,20-05-2011,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,07-06-2019,Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,01-05-2015,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,15-12-2017,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


### Cleaning Movie Database

In [ ]:
#drop null values and confirm the null values don't exist
movie_database=movie_database.dropna()
movie_database.isnull().sum()

movie_id           0
primary_title      0
genres             0
start_year         0
runtime_minutes    0
averagerating      0
numvotes           0
dtype: int64

In [ ]:
movie_database.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65720 entries, 0 to 73051
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         65720 non-null  object 
 1   primary_title    65720 non-null  object 
 2   genres           65720 non-null  object 
 3   start_year       65720 non-null  int64  
 4   runtime_minutes  65720 non-null  float64
 5   averagerating    65720 non-null  float64
 6   numvotes         65720 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 4.0+ MB


In [ ]:
#rename primary_title to movie_tile and drop the movie_id column
movie_database.rename(columns={'primary_title':'movie_title'},inplace=True)
movie_database.drop(columns='movie_id',inplace=True)

### Merging DataFrames
Combine the two dataframes into a single dataframe to enable analysis

In [ ]:
# merge movie_budgets and movie_database
merged_movies=pd.merge(movie_budgets,movie_database,on=['movie_title'],how='inner')

In [91]:
merged_movies.head()

,release_date,movie_title,production_budget,domestic_gross,worldwide_gross,genres,start_year,runtime_minutes,averagerating,numvotes
0,18-12-2009,Avatar,425000000.0,760507625.0,2.776345e+09,Horror,2011,93.0,6.1,43
1,20-05-2011,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,"Action,Adventure,Fantasy",2011,136.0,6.6,447624
2,07-06-2019,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,"Action,Adventure,Sci-Fi",2019,113.0,6.0,24451
3,01-05-2015,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,"Action,Adventure,Sci-Fi",2015,141.0,7.3,665594
4,27-04-2018,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,"Action,Adventure,Sci-Fi",2018,149.0,8.5,670926


### New Feature
Generate a new feature called revenue that measures profit of each movie.This will show how each film profite based on the **production_budget** and **worldwide_gross**

In [98]:
#create profit column
merged_movies['Revenue'] = merged_movies['worldwide_gross']-merged_movies['production_budget']

In [100]:
#reorder the columns
merged_movies[['movie_title','genres','production_budget','worldwide_gross','Revenue','domestic_gross','start_year',
                'release_date','runtime_minutes','averagerating','numvotes']]

,movie_title,genres,production_budget,worldwide_gross,Revenue,domestic_gross,start_year,release_date,runtime_minutes,averagerating,numvotes
0,Avatar,Horror,425000000.0,2.776345e+09,2.351345e+09,760507625.0,2011,18-12-2009,93.0,6.1,43
1,Pirates of the Caribbean: On Stranger Tides,"Action,Adventure,Fantasy",410600000.0,1.045664e+09,6.350639e+08,241063875.0,2011,20-05-2011,136.0,6.6,447624
2,Dark Phoenix,"Action,Adventure,Sci-Fi",350000000.0,1.497624e+08,-2.002376e+08,42762350.0,2019,07-06-2019,113.0,6.0,24451
3,Avengers: Age of Ultron,"Action,Adventure,Sci-Fi",330600000.0,1.403014e+09,1.072414e+09,459005868.0,2015,01-05-2015,141.0,7.3,665594
4,Avengers: Infinity War,"Action,Adventure,Sci-Fi",300000000.0,2.048134e+09,1.748134e+09,678815482.0,2018,27-04-2018,149.0,8.5,670926
...,...,...,...,...,...,...,...,...,...,...,...
2747,Stories of Our Lives,Drama,15000.0,0.000000e+00,-1.500000e+04,0.0,2014,31-12-2014,60.0,7.5,85
2748,Cure,Drama,10000.0,9.459600e+04,8.459600e+04,94596.0,2011,06-07-2001,93.0,4.9,11
2749,Newlyweds,"Comedy,Drama",9000.0,4.584000e+03,-4.416000e+03,4584.0,2011,13-01-2012,95.0,6.4,1436
2750,Red 11,"Horror,Sci-Fi,Thriller",7000.0,0.000000e+00,-7.000000e+03,0.0,2019,31-12-2018,77.0,5.6,43


In [101]:
merged_movies.head()

,release_date,movie_title,production_budget,domestic_gross,worldwide_gross,genres,start_year,runtime_minutes,averagerating,numvotes,Revenue
0,18-12-2009,Avatar,425000000.0,760507625.0,2.776345e+09,Horror,2011,93.0,6.1,43,2.351345e+09
1,20-05-2011,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,"Action,Adventure,Fantasy",2011,136.0,6.6,447624,6.350639e+08
2,07-06-2019,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,"Action,Adventure,Sci-Fi",2019,113.0,6.0,24451,-2.002376e+08
3,01-05-2015,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,"Action,Adventure,Sci-Fi",2015,141.0,7.3,665594,1.072414e+09
4,27-04-2018,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,"Action,Adventure,Sci-Fi",2018,149.0,8.5,670926,1.748134e+09


In [102]:
#save the clean data into a csv file
merged_movies.to_csv('merged_movies.csv',index=False)